In [ ]:
class color:
    BOLD = '\033[1m'
    END = '\033[0m'

In [ ]:
%matplotlib inline
import os
import numpy as np
from datetime import datetime
from qiskit.dagcircuit import DAGCircuit
from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer

from qiskit.circuit import QuantumCircuit, QuantumRegister

from IPython.display import Image

from CircuitGeneration.circuitGenerator import createTestFile

def printDag(circ):
    dag = circuit_to_dag(circ)
    dag_drawer(dag, filename="./example.png")
    display(Image(filename='example.png') )
    
# import passmanagers
from Passmanager.passmanager_SA import testing_pass_manager
from Passmanager.passmanager_Init import testing_pass_manager_init
from Passmanager.passmanager_Lvl3 import level_3_pass_manager_simplyfied
from Passmanager.passmanager_Lvl3_pure import level_3_pass_manager_pure

def logConfiguration(logPath, configs):
    with open(logPath, "a") as f:
        f.write(configs + "\n")
        f.close()

# Passmanager creation and evaluation helper

In [ ]:
import csv
import json

import matplotlib.pyplot as plt

def evalCircuitRuns(logFiles, showPlots=True, tableMode=False):
    maxOpt = 0
    best = "None"
    for i, log in enumerate(logFiles):
        print(color.BOLD + "Evaluation " + str(i+1) + color.END)
        totalGates = []
        totalCX = []
        totalX = []
        totalSX = []
        totalRZ = []
        with open(log) as log:
            csv_reader = csv.reader(log, delimiter=';')
            info = next(csv_reader)
            print(info)
            for row in csv_reader:
                totalGates.append(int(row[0]))
                jsonDict = json.loads(row[1].replace("\'", "\""))
                if "cx" in jsonDict :
                    totalCX.append(jsonDict["cx"])
                else:
                    totalCX.append(0)
                if "x" in jsonDict :
                    totalX.append(jsonDict["x"])
                else:
                    totalX.append(0)
                if "sx" in jsonDict :
                    totalSX.append(jsonDict["sx"])
                else:
                    totalSX.append(0)
                if "rz" in jsonDict :
                    totalRZ.append(jsonDict["rz"])
                else:
                    totalRZ.append(0)
        
        ## Analyse:
        print("Total gate reduction:", str(totalGates[1] - totalGates[-1]), "(" +  str(100*(totalGates[1] - totalGates[-1])/totalGates[1]) + "%)")
        if totalGates[1] - totalGates[-1] > maxOpt:
            maxOpt = totalGates[1] - totalGates[-1]
            best = info
        if totalCX[1] != 0 : 
            print("CX-gate reduction:", str(totalCX[1] - totalCX[-1]), "(" +  str(100*(totalCX[1] - totalCX[-1])/totalCX[1]) + "%)")
        else:
            print("CX-gate reduction:", str(totalCX[1] - totalCX[-1]), "(" +  str(0) + "%)")
            
        if not tableMode:
            if totalX[1] != 0 : 
                print("X-gate reduction:", str(totalX[1] - totalX[-1]), "(" + str(100*(totalX[1] - totalX[-1])/totalX[1]) + "%)")
            else:
                print("X-gate reduction:", str(totalX[1] - totalX[-1]), "(" + str(0) + "%)")

            if totalSX[1] != 0 : 
                print("SX-gate reduction:", str(totalSX[1] - totalSX[-1]), "(" +  str(100*(totalSX[1] - totalSX[-1])/totalSX[1]) + "%)")
            else:
                print("SX-gate reduction:", str(totalSX[1] - totalSX[-1]), "(" +  str(0) + "%)")

            if totalRZ[1] != 0 : 
                print("RZ-gate reduction:", str(totalRZ[1] - totalRZ[-1]), "(" +  str(100*(totalRZ[1] - totalRZ[-1])/totalRZ[1]) + "%)")
            else:
                print("RZ-gate reduction:", str(totalRZ[1] - totalRZ[-1]), "(" + str(0) + "%)")
        
        
        if showPlots:
            # Plot
            plt.figure()
            yint = range(len(totalGates))

            plt.plot(yint, totalGates, label="total")
            plt.plot(yint, totalCX, label="CX", linestyle=':')
            plt.plot(yint, totalX, label="X", linestyle=':')
            plt.plot(yint, totalSX, label="SX", linestyle=':')
            plt.plot(yint, totalRZ, label="RZ", linestyle=':')

            plt.legend()
            plt.xlabel("Iterations")
            plt.ylabel("# Gates")
            plt.xticks(yint)
            plt.show()
    if not tableMode:
        print(str(best) + "\n" + "with total opt " + str(maxOpt))

In [ ]:
def runPassmanagers(circ, circuitName):
    ops = circ.count_ops()
    if "cx" in ops:
        cx = ops['cx']
    else:
        cx = 1
    # List 
    print("preparing passmanagers")
    logFiles = []

    passmanagers = []

    # PM init
    managerName = "SA_init_multiIt"
    logFile = "./logs/" + circuitName + "_" + managerName + "_" + str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S")) + ".log"
    pm = testing_pass_manager_init(logFile=logFile)
    passmanagers.append(pm)
    logFiles.append(logFile)
    logConfiguration(logFile, "SA_init, alpha = 0.8")
    
    # One step
    probabilities = [0, 1]    
    a = 0.9
    n = 10
    iT = 15
    minTemp = 0.00000125
    # PM step 1.1
    managerName = "1SA-" + str(len(probabilities)-1) + "Bit_multiIt"
    logFile = "./logs/" + circuitName + "_" + managerName + "_" + str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S")) + ".log"   
    pm = testing_pass_manager( probabilities, n=n, initTemp = iT, logFile=logFile, minTemp=minTemp, singleIt=False, alpha=a)
    passmanagers.append(pm)
    logFiles.append(logFile)
    logConfiguration(logFile, managerName + " probabilities:" + str(probabilities) + ", alpha = " + str(a) + " initTemp=" + str(iT) + " minTemp=" + str(minTemp))
    
    # two steps
    probabilities = [0, 0.6, 0.4]
    a = 0.9
    n = 10
    iT = 15
    minTemp = 0.00000125
    # PM step 1.1
    managerName = "1SA-" + str(len(probabilities)-1) + "Bit_multiIt"
    logFile = "./logs/" + circuitName + "_" + managerName + "_" + str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S")) + ".log"   
    pm = testing_pass_manager( probabilities, n=n, initTemp = iT, logFile=logFile, minTemp=minTemp, singleIt=False, alpha=a)
    passmanagers.append(pm)
    logFiles.append(logFile)
    logConfiguration(logFile, managerName + " probabilities:" + str(probabilities) + ", alpha = " + str(a) + " initTemp=" + str(iT) + " minTemp=" + str(minTemp))
    
    # Passmanager Lvl3 basic
    managerName = "Lvl3"
    logFile = "./logs/" + circuitName + "_" + managerName + "_" + str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S")) + ".log"
    pm = level_3_pass_manager_simplyfied(logFile=logFile)
    passmanagers.append(pm)
    logFiles.append(logFile)
    logConfiguration(logFile,"Passmanager lvl3, CommutativeCancellation")
    
    # Passmanager Lvl3 full
    managerName = "Lvl3-pure"
    logFile = "./logs/" + circuitName + "_" + managerName + "_" + str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S")) + ".log"
    pm = level_3_pass_manager_pure(logFile=logFile)
    passmanagers.append(pm)
    logFiles.append(logFile)
    logConfiguration(logFile,"Passmanager lvl3, No CommutativeCancellation")

    # Execute Passmanagers
    print(len(passmanagers),"passmanagers found...\nstarting optimizations")
    for i, manager in enumerate(passmanagers):
        print("Starting PM", i+1, "of", len(passmanagers))
        new_circ = manager.run(circ)
        print("Finished PM", i+1)
    print("Optimizations finisched")
    return logFiles

In [ ]:
def runPassmanagers_singleRun(circ, circuitName):
    ops = circ.count_ops()
    if "cx" in ops:
        cx = ops['cx']
    else:
        cx = 1
    # List 
    print("preparing passmanagers")
    logFiles = []

    passmanagers = []
    # PM init
    managerName = "SA_init_singleIt"
    logFile = "./logs/" + circuitName + "_" + managerName + "_" + str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S")) + ".log"
    pm = testing_pass_manager_init(singleIt=True, logFile=logFile)
    passmanagers.append(pm)
    logFiles.append(logFile)
    logConfiguration(logFile, "SA_init, single Iteration")
    
    # One step
    probabilities = [0, 1]    
    a = 0.9
    n = 10
    iT = 15
    minTemp = 0.00000125
    managerName = "1SA-" + str(len(probabilities)-1) + "Bit_singleIt"
    logFile = "./logs/" + circuitName + "_" + managerName + "_" + str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S")) + ".log"   
    pm = testing_pass_manager( probabilities, n=n, initTemp = iT, logFile=logFile, minTemp=minTemp, singleIt=True, alpha=a)
    passmanagers.append(pm)
    logFiles.append(logFile)
    logConfiguration(logFile, managerName + " probabilities:" + str(probabilities) + ", alpha = " + str(a) + " initTemp=" + str(iT) + " minTemp=" + str(minTemp))
    
    
    # two steps
    probabilities = [0, 0.6, 0.4]
    a = 0.9
    n = 10
    iT = 15
    minTemp = 0.00000125
    managerName = "1SA-" + str(len(probabilities)-1) + "Bit_singleIt"
    logFile = "./logs/" + circuitName + "_" + managerName + "_" + str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S")) + ".log"   
    pm = testing_pass_manager( probabilities, n=n, initTemp = iT, logFile=logFile, minTemp=minTemp, singleIt=True, alpha=a)
    passmanagers.append(pm)
    logFiles.append(logFile)
    logConfiguration(logFile, managerName + " probabilities:" + str(probabilities) + ", alpha = " + str(a) + " initTemp=" + str(iT) + " minTemp=" + str(minTemp))
       
    # Execute Passmanagers
    print(len(passmanagers),"passmanagers found...\nstarting optimizations")
    for i, manager in enumerate(passmanagers):
        print("Starting PM", i+1, "of", len(passmanagers))
        new_circ = manager.run(circ)
        print("Finished PM", i+1)
    print("Optimizations finisched")
    return logFiles

# Example optimization

In [ ]:
circName = "example1.qasm"
folder = "./SmallCircuits/"
circ = QuantumCircuit.from_qasm_file(folder+circName)

logFiles = runPassmanagers(circ, "testFile")

In [ ]:
evalCircuitRuns(logFiles)

# Small circuits

In [ ]:
runs = []
folder = "./SmallCircuits/"
for filename in os.listdir(folder):
    print(color.BOLD + "Optimizing " + filename + color.END)
    circ = QuantumCircuit.from_qasm_file(folder + filename)
    logFiles = runPassmanagers(circ, filename)
    runs.append((logFiles, filename))
print("Finished all qasm files in folder...")

In [ ]:
for run in runs:
    logFiles, circuitName = run
    print(color.BOLD + "Evaluating circuit " + circuitName + color.END)
    evalCircuitRuns(logFiles, showPlots=False, tableMode=True)
print("Finished all runs...")

# Large circuits single iteration

In [ ]:
runs = []
folder = "./LargeCircuits/"
for filename in os.listdir(folder):
    print(color.BOLD + "Optimizing " + filename + color.END)
    circ = QuantumCircuit.from_qasm_file(folder + filename)
    logFiles = runPassmanagers_singleRun(circ, filename)
    runs.append((logFiles, filename))
print("Finished all qasm files in folder...")

In [ ]:
for run in runs:
    logFiles, circuitName = run
    print(color.BOLD + "Evaluating circuit " + circuitName + color.END)
    evalCircuitRuns(logFiles, showPlots=False, tableMode=True)
print("Finished all runs...")

# Large circuits full optimization

In [ ]:
runs = []
folder = "./LargeCircuits/"
for filename in os.listdir(folder):
    #with open(os.path.join("./circuits/", filename), 'r') as f: # open in readonly mode
    #print("Optimizing", filename)
    print(color.BOLD + "Optimizing " + filename + color.END)
    # do your stuff
    circ = QuantumCircuit.from_qasm_file(folder + filename)
    logFiles = runPassmanagers(circ, filename)
    runs.append((logFiles, filename))
print("Finished all qasm files in folder...")

In [ ]:
for run in runs:
    logFiles, circuitName = run
    print(color.BOLD + "Evaluating circuit " + circuitName + color.END)
    evalCircuitRuns(logFiles, showPlots=False, tableMode=True)
print("Finished all runs...")